# Notes
## Authorization
* Need to use `client_id` and `client_secret` to retrieve a auth token that will allow us to interact with the Spotify API service.
* Typically authenticate once, retrieve a token which is then attached to your session. During your current session you'll be able to interact/make multiple requests with the service. This allows you to not have to login every time to access your session.
* Handy [Authorization Flow Guide](https://developer.spotify.com/documentation/general/guides/authorization-guide/) from Spotify. We are buidling a **client** to interact with Spotify's API. You have to authenticate how a *user* authenticates with your *client* **and** Spotify at the same time (advanced - not covered fully in this tutorial). We're only using the **Client Credentials** flow, which requires a `Client ID` and `Secret Key` and we'll get in return an `Access Token`. This approach won't give give access to manage private user data, just general data only.
* Our **client** is the application we're building and it (the app) needs to request authorization by sending a `POST` request.
* `base64` is a more secure string format. For non-Base 64 encoded strings, you generally can build something like `client_creds = f"{client_id}:{client_secret}"`. You can encode/decode `str` to bytes using `str.encode()` and `str.encode().decode()`. The Spotify backend API is going to take our base64 encoded `str` (not `bytes`) we pass and then `base64.base64decode()` to verify credentials. Steps to do this:
    1. First take `str` and encode to `bytes` using `str.encode()`
    2. Then encode this bytestring to base64 using `base64.b64encode(bytestring)`
    3. Next, decode this base64 `bytes` so it's back to `str` using `client_creds_b64.decode()` (**not** `b64decode()`).
* The token response data we get after making the `POST` request to authorize our client has the `access_token` and the `expires_in` (seconds) details. We can use `datetime` and the `timedelta(seconds=expires_in)` to create an easy `expires` `datetime.datetime` object for tracking. We can check whether our token expired and to fetch a new one: `did_expire: bool = expires < now`.
## Base API Client Class
* Once we have our `access_token` then we can use it. However, the `expires` and `access_tokens` are **state-like** items, so they will change over time. So, better would be to create a `BaseClient` **class** that can adjust/respond to these state changes in order to make our API calls through the class (and its methods) rather than just through simple functions like we did in `Auth.ipynb`. We're going to turn our functions into class methods.
* Create our first class `SpotifyAPI(object)` with some general-purpose variables (nothing hard-coded since we want to reuse this for various sessions). We can pass the `client_id` and `client_secret` to the `__init__` function. The `super().__init__(*args, **kwargs)` allows us to call the class our class is inheriting from *directly*. Currently we're not doing that but in the future if we wanted to then having this line allows us to do that. 
* ?? We set `client_id` and `client_secret` to `None` by default for the class. However, we need to **update** them...?? Aren't these already available though since we set them inside the __init__ function? I think so but this makes it cleaner so we don't have to `{self.client_id}`? Not sure though as he mentioned "updating" these values...
    * I *believe* we have to update the class variables within these methods because when we initialize/create a new instance of the class, we only pass in `client_id` and `client_secret` (based on our `__init__`). However, the other class variables (`access_token`, `expires`, etc.) are defaulted to `None` so our class methods need to update these values e.g, `self.access_token = access_token` (once we have retrieved the true `access_token`).
* **Refactor** Could refactor the base class to have a method that handles the response data instead of all within the current `perform_authorization()` method.
* **Refactor** Need to improve how we get the `access_token` because when it does expire, ideally it will go fetch another one automatically by re-performing this authorization function again.
* Eventually going to take this base class and expand it to allow for something like `client.search` (`spotify.search`).
https://developer.spotify.com/documentation/web-api/reference/

In [3]:
import typing as t
import requests
import base64
import datetime

In [4]:
# Snag creds from developer.spotify.com dashboard
client_id: str
client_secret: str

In [5]:
# Declare our first class with helper methods
class SpotifyAPIClient(object):
    access_token: str = None
    access_token_expires = datetime.datetime.now()  # or None
    access_token_did_expire: bool = True
    client_id: str = None
    client_secret: str = None
    token_url: str = "https://accounts.spotify.com/api/token"

    def __init__(self, client_id, client_secret, *args, **kwargs):
        # Call super() so we can call any class we inherit from directly!
        super().__init__(*args, **kwargs)
        self.client_id = client_id
        self.client_secret = client_secret

    def get_client_credentials(self) -> str:
        """
        Returns a Base 64 encoded string (not bytes!)
        """
        # Need to declare our client_id and client_secret vars
        # The self obj has them but need to update within this
        # method's scope:
        # NOTE ?? Aren't these already available though since we
        # set them inside the __init__ function? I think so but
        # this makes it cleaner so we don't have to {self.client_id}??
        client_id: str = self.client_id
        client_secret: str = self.client_secret

        # Update the client_id and client_secret values
        if client_id is None or client_secret is None:
            raise Exception("You must set client_id and client_secret.")

        # Build the credentials str required by Spotify:
        client_creds: str = f"{client_id}:{client_secret}"
        client_creds_bytes: bytes = client_creds.encode()
        client_creds_b64: bytes = base64.b64encode(client_creds_bytes)
        # NOTE We must .decode() back to 'str' type instead of 'bytes'
        client_creds_b64_decoded: str = client_creds_b64.decode()

        return client_creds_b64_decoded

    def get_token_headers(self) -> t.Dict:
        """
        Pass 'Authorization' header with b64 encoded creds
        "Authorization": "Basic <base64 encoded client_id:client_secret>"
        """
        client_creds_b64_decoded: str = self.get_client_credentials()

        token_headers: t.Dict = {
            "Content-Type": "application/x-www-form-urlencoded",
            "Authorization": f"Basic {client_creds_b64_decoded}"
        }

        return token_headers
    
    def get_token_data(self) -> t.Dict:
        token_data: t.Dict = {
            "grant_type": "client_credentials",
        }

        return token_data

    def perform_authorization(self) -> bool:
        """
        Extracts the access token and other variables for authorizing
        the client app with Spotify API. It uses helper methods to get
        client credentials, convert to Base 64, get token headers and
        token data. 
        """
        # TODO Re-run authorization if access_token expires
        # Use helper methods to retrieve components for sending request
        token_url: str = self.token_url
        token_data: t.Dict = self.get_token_data()
        token_headers: t.Dict = self.get_token_headers()

        # Now we have everything for authentication so it's time to make the POST request
        r = requests.post(token_url, data=token_data, headers=token_headers)

        if r.status_code not in range(200, 299):
            print(f"Failed request! {r.status_code}")
            return False

        # Let's store the access token, expires in (seconds), etc.
        data: t.Dict = r.json()
        now = datetime.datetime.now()
        access_token: str = data['access_token']
        expires_in: float = data['expires_in']
        expires = now + datetime.timedelta(seconds=expires_in)

        # Update our class variables with these updated values
        self.access_token = access_token
        self.access_token_expires = expires  # datetime.datetime obj
        self.access_token_did_expire = expires < now  # refetch the token if True

        return True  # authorization successful


In [7]:
# Now let's test out our base class (client app)
# client = SpotifyAPI(client_id, client_secret)
# Could generalize this client for multiple services
spotify = SpotifyAPIClient(client_id, client_secret)

In [8]:
# Perform authorization
spotify.perform_authorization()

True

In [9]:
spotify.access_token

'BQDz7lDDe7xepdSMHC5iMgeuB3O_z0lBJ-buUaWqFB1NlmxvOMpcjZUaBzx-_qmsKFwNWuC-9oLxPtSA2UQ'